In [5]:
import string
import re
from bs4 import BeautifulSoup
import pandas as pd
import requests
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import syllables
from pandas import DataFrame

In [8]:
input = pd.read_excel("./Input.xlsx")
input.head()

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


In [18]:
import string
import re
from bs4 import BeautifulSoup
import pandas as pd
import requests
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import syllables
from pandas import DataFrame
import os
nltk.download('punkt')
nltk.download('stopwords')
# path = "G:\\blackcoffer\\articles\\"
url_input = pd.read_excel("Input.xlsx")
stem = PorterStemmer()


def steaming(content):
    content = word_tokenize(content)
    w = []
    for word in content:
        w.append(stem.stem(word))
    return list(set(w))


def syllable_count(content):
    syllables_len = []
    content = steaming(content)
    for word in content:
        syllables_len.append(syllables.estimate(word))
    return syllables_len


def total_syllable(content):
    total = 0
    syllables_len = syllable_count(content)
    for i in syllables_len:
        total = total + i
    return total


def complex_word_count(content):
    syllables_len = syllable_count(content)
    complex_word: int = 0
    for i in syllables_len:
        if i > 2:
            complex_word += 1
    return complex_word


def read_article_file(file_name="./articles/blackassign0001.txt", type_="utf-8"):
    try:
        with open(file_name, "r", encoding=type_) as f:
            return f.read()
    except:
        return ""

def remove_stopWord(content, type_="len"):
    content = content.translate(str.maketrans('', '', string.punctuation))
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(content)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    if type_ == "word":
        return filtered_sentence
    else:
        return len(filtered_sentence)


def master_dictionary():
    negative = read_article_file("./MasterDictionary/negative-words.txt", None)
    negative = word_tokenize(negative)
    positive = word_tokenize(read_article_file("./MasterDictionary/positive-words.txt", None))
    stop_words = set(stopwords.words('english'))
    negative = [w for w in negative if not w.lower() in stop_words]
    positive = [w for w in positive if not w.lower() in stop_words]
    master_dict = {
        "positive": positive,
        "negative": negative
    }
    return master_dict


def positive_score(content):
    count: int = 0
    master_dict = master_dictionary()
    content = remove_stopWord(content, 'word')
    for pos in master_dict['positive']:
        if pos in content:
            count += 1
    return count


def negative_score(content):
    count: int = 0
    master_dict = master_dictionary()
    content = remove_stopWord(content, 'word')
    for neg in master_dict['negative']:
        if neg in content:
            count -= 1
    return (count * (-1))


def polarity_score(content):
    positive = positive_score(content)
    negative = negative_score(content)
    polarity = (positive - negative) / ((positive + negative) + 0.000001)
    return polarity


def subjective_score(content):
    positive = positive_score(content)
    negative = negative_score(content)
    total_words = remove_stopWord(content)
    score = (positive + negative) / (total_words + 0.000001)
    return score


def scrap_data():
    input = pd.read_excel("./Input.xlsx")
    input.head()
    
    for i in range(0, input.shape[0]-1):
        url_id = input.iloc[i,0]
        url = input.iloc[i,1]
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            article = soup.getText(separator=" ", strip= True)
            os.makedirs(os.path.dirname(f"articles/{url_id}.txt"), exist_ok=True)

            with open(f"articles/{url_id}.txt", "w", encoding="utf-8") as file:
                file.write(article)

def personal_pronouns(content):
    per_pro_list = re.findall('i|I|We|we|My|my|Ours|ours|our|Our|us|Us', content)
    return len(per_pro_list)

def avg_word_sent(content):
    try:
        sentence = sent_tokenize(content)
        words = word_tokenize(content)
        avg_word = round(len(words) / len(sentence), 2)
    except ZeroDivisionError:
        avg_word = 0
    return avg_word


def avg_sent_len(content):
    sentence = sent_tokenize(content)
    words = word_tokenize(content)
    try:
        return round(len(words) / len(sentence), 2)
    except ZeroDivisionError:
        return 0


def percentage_complex_word(content):
    complex_count: int = complex_word_count(content)
    words = word_tokenize(content)
    try:
        perc = round((complex_count / len(words)) * 100, 2)
    except ZeroDivisionError:
        perc = 0
    return perc


def fog_index(content):
    complex = percentage_complex_word(content)
    sentence = avg_sent_len(content)
    index = 0.4 * (sentence + complex)
    return index


def avg_word_length(content):
    try:
        words = word_tokenize(content)
        word_len = len(words)
        char_count = 0
        for i in words:
            for count in i:
                if count.isalpha():
                    char_count += 1
        avg_word_count = round(char_count / word_len, 2)
    except ZeroDivisionError:
        avg_word_count = 0
    return avg_word_count


if __name__ == "__main__":
    try:
        output: DataFrame = pd.read_excel("Output Data Structure.xlsx")
        pd.set_option('mode.chained_assignment', None)
    except FileNotFoundError:
        print("Can't open file ")
        exit()
    for index in output.index:
        content = read_article_file(f'./articles/{output["URL_ID"][index]}.txt')
        output['POSITIVE SCORE'][index] = positive_score(content)
        output['NEGATIVE SCORE'][index] = negative_score(content)
        output['POLARITY SCORE'][index] = polarity_score(content)
        output['SUBJECTIVITY SCORE'][index] = subjective_score(content)
        output["AVG WORD LENGTH"][index] = avg_word_length(content)
        output['PERSONAL PRONOUNS'][index] = personal_pronouns(content)
        output['COMPLEX WORD COUNT'][index] = complex_word_count(content)
        output['PERCENTAGE OF COMPLEX WORDS'][index] = percentage_complex_word(content)
        output['FOG INDEX'][index] = fog_index(content)
        output['AVG NUMBER OF WORDS PER SENTENCE'][index] = avg_word_sent(content)
        output['SYLLABLE PER WORD'][index] = len(syllable_count(content))
        output['PERSONAL PRONOUNS'][index] = personal_pronouns(content)
        output['AVG SENTENCE LENGTH'][index] = avg_sent_len(content)
        output['WORD COUNT'][index] = avg_word_length(content)
        
    output.to_csv("output.csv")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Krishna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Krishna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
!pip install nltk["punkt"]


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\Krishna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
